In [13]:
import mbuild as mb 

import numpy as np 


from pathlib import Path
import sys
directory = Path('/Users/kieran/surface_coatings')
sys.path.append(str(directory))

# from surface_coatings import surface_coatings 
from surface_coatings.surfaces import SilicaInterfaceCarve, SiliconInterface, SilicaInterface
from surface_coatings.chains import SilanePolymer, VBCPolymer

from surface_coatings import Monolayer
from mbuild.lib.atoms import H
from surface_coatings.chains import SilanePolymer
from surface_coatings.monomers.mpc import MPC
from mbuild.lib.moieties import Silane

from surface_coatings import SolvatedDualMonolayer, DualMonolayer
import foyer 
from foyer import Forcefield

from gmso.external.convert_mbuild import to_mbuild
from foyer import general_forcefield

from utils.index_groups import generate_index_groups

In [2]:
monomer = SilanePolymer(n=1)
# display(monomer.visualize(backend='nglview',show_ports=True))
comps = list(monomer.particles())
monomer.remove([comps[0],comps[2],comps[3],comps[36]])
first_h = H()
second_h = H()
monomer.add(first_h)
monomer.add(second_h)
mb.force_overlap(first_h, first_h['up'], monomer['down'])
mb.force_overlap(second_h, second_h['up'], monomer.available_ports()[0])
display(monomer.visualize(backend='nglview'))
# monomer.save('length_1_mpc_silane_monomer_h_capped.mol2')
# silane = mb.load('[SiH4]',smiles=True)
# comps = list(silane.particles())
# silane.remove([comps[1],comps[2]])

/Users/kieran/mbuild/mbuild/conversion.py:481: UserWarning: Using parmed reader. Bonds may be inferred from inter-particle distances and standard residue templates. Please check that the bonds in mb.Compound are accurate
  warn(


NGLWidget()

In [16]:
test = mb.load('silane_pmpc.xyz')
for part in test.particles():
    if "Si" in part.name:
        part.element = 'Si'
    else:
        part.element = part.name[0]
test2 = mb.load('test.pdb', backend='parmed')
display(test.visualize(backend='nglview'))
bonds = list(test2.bonds())
for b in bonds:
    if test2[0] in b:
        for part in b:
            if part.element.symbol == 'H':
                bond_to_remove = b
                break
test2.remove_bond(bond_to_remove)
display(test2.visualize(backend='nglview'))
# test2.save('silane_pmpc_moner_fixed.mol2',overwrite=True)

NGLWidget()

NGLWidget()

In [10]:
surface = SilicaInterfaceCarve()
chain = SilanePolymer(n=2)
chain.visualize()

/Users/kieran/mbuild-examples/mbuild/mbuild/compound.py:755: UserWarning: The Compound you are adding has a box. The box of the parent compound will be used. Use inherit_box = True if you wish to replace the parent compound box with that of Compound being added.
  warn(
/Users/kieran/mbuild-examples/mbuild/mbuild/utils/decorators.py:134: Warning: freud_generate_bonds is an experimental feature and is not subject to follow standard deprecation cycles. Use at your own risk! 
  warn(


<MPC 41 particles, non-periodic, 40 bonds, id: 140381666472912>
[<Port, anchor: 'C', labels: , id: 140381667781312>, <Port, anchor: 'C', labels: , id: 140381667811632>]


/Users/kieran/mbuild-examples/mbuild/mbuild/conversion.py:481: UserWarning: Using parmed reader. Bonds may be inferred from inter-particle distances and standard residue templates. Please check that the bonds in mb.Compound are accurate
  warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
monolayer = Monolayer(surface=mb.clone(surface), 
                      chains=mb.clone(chain), 
                      n_chains=20, 
                      tile_x=1, 
                      tile_y=1)
dual_monolayer = DualMonolayer(top=mb.clone(monolayer),
                               bottom=mb.clone(monolayer),
                               separation=3,
                               shift=True)
solvated_dual_monolayer = SolvatedDualMonolayer(dual_monolayer=mb.clone(dual_monolayer))

/Users/kieran/surface_coatings/surface_coatings/monolayer.py:84: UserWarning: 
 No fractions provided. Assuming a single chain type.
  warn("\n No fractions provided. Assuming a single chain type.")
/Users/kieran/mbuild-examples/mbuild/mbuild/compound.py:768: UserWarning: After adding new Compound, Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box
  warn(
/Users/kieran/mbuild-examples/mbuild/mbuild/compound.py:1172: UserWarning: Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box.
  warn(


In [12]:
oplsaa = Forcefield(forcefield_files="./utils/charge_neutral_oplsaa_imodels.xml")
typed_monolayer = oplsaa.apply(chain, verbose=True,
                          assert_angle_params=False,
                          assert_dihedral_params=False)
typed_monolayer.save('solvated_dual_monolayer.top', overwrite=True)

/Users/kieran/opt/miniconda3/lib/python3.9/site-packages/foyer/validator.py:165: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


Missing angle with ids (42, 45, 46) and types ['opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (1, 78, 42, 45) and types ['opls_139', 'opls_136', 'opls_141', 'opls_465'].
Missing dihedral with ids (4, 1, 78, 42) and types ['opls_465', 'opls_139', 'opls_136', 'opls_141'].


/Users/kieran/opt/miniconda3/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all angles. Total system angles: 158, Parameterized angles: 157
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 201, Parameterized dihedrals: 199. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 96, Parameterized impropers: 3. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/Users/kieran/opt/miniconda3/lib/python3.9/site-packages/foyer/for

In [14]:
index_groups = generate_index_groups(system=typed_monolayer,
					 terminal_groups=terminal_groups,
					 freeze_thickness=0.5)

NameError: name 'terminal_groups' is not defined